## Load Routines for Printing Matrices

In [1]:
import Formatting              #do Pkg.add("Formatting") to install this package
include("printmat.jl")

printmat (generic function with 4 methods)

# Covariance Matrix from Betas

Recall: the single index model implies that $\operatorname*{Cov}(R_{t})=\beta\beta^{\prime}\operatorname*{Var}
(R_{mt})+\Sigma$

In [2]:
β      = [0.9;1.1]
StdRes = [10;5]
VarRm  = 4^2
Σ      = diagm(StdRes.^2)                  #correlation matrix -> covariance matrix 
printmat(round(Σ,3))

CovR = β*β'*VarRm + Σ
println("Covariance matrix calculated from betas: ")
printmat(round(CovR,2))

   100.000     0.000
     0.000    25.000

Covariance matrix calculated from betas: 
   112.960    15.840
    15.840    44.360



# Estimating a Single-Index Model

In [3]:
xx   = readdlm("Data/FFmFactorsPs.csv",',',header=true)
x    = xx[1]
Rme  = x[:,2]                #market excess return
RSMB = x[:,3]                #small minus big firms
RHML = x[:,4]                #high minus low book-to-market ratio
Rf   = x[:,5]                #interest rate


x  = readdlm("Data/FF25Ps.csv",',') #no header line: x is matrix
R  = x[:,2:end]                     #returns for 25 FF portfolios
Re = R .- Rf                        #excess returns for the 25 FF portfolios
Re = Re[:,[1;7;13;19;25]]           #use just 5 assets to make the printing easier 

(T,n) = size(Re)                    #no. obs and  no. test assets
#-------------------------------------------------------------------------

c      = ones(T,1)                 #a vector with ones
x      = [c Rme]                   #regressors
β      = fill(NaN,n)
VarRes = fill(NaN,n)
for i = 1:n
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    β[i]      = b_i[2]
    VarRes[i] = var(residual)
end    
println("\nOLS coefficients, regressing Re on constant and Rme")
printmat(round(β,3)) 
#-------------------------------------------------------------------------

Σ     = diagm(VarRes)             #creates a diagonal covariance matrix
VarRm = var(Rme)
CovR  = β*β'*VarRm + Σ
println("\nCovariance matrix calculated from betas: ")
printmat(round(CovR,2))

println("\nCovariance matrix calculated from data: ")
printmat(round(cov(Re),2))


OLS coefficients, regressing Re on constant and Rme
     1.341
     1.169
     0.994
     0.943
     0.849


Covariance matrix calculated from betas: 
    73.480    33.230    28.270    26.800    24.150
    33.230    37.370    24.640    23.360    21.050
    28.270    24.640    26.800    19.870    17.910
    26.800    23.360    19.870    25.200    16.970
    24.150    21.050    17.910    16.970    25.330


Covariance matrix calculated from data: 
    73.480    43.020    29.470    25.010    20.190
    43.020    37.370    29.070    25.140    21.410
    29.470    29.070    26.800    23.630    20.980
    25.010    25.140    23.630    25.200    21.530
    20.190    21.410    20.980    21.530    25.330



# A Multi-Index Model

Recall: if $\Omega$ is the covariance matrix of the indices, then the covariance of
assets $i$ and $j$ is

$\sigma_{ij}=b_{i}^{\prime}\Omega b_{j}$

where $b_{i}$ is the vector of slope coefficients obtained from regressing
$R_{it}$ on the vector of factors

In [4]:
x      = [c Rme RSMB RHML]               #regressors
K      = size(x,2) - 1 
b      = fill(NaN,(K,n))
VarRes = fill(NaN,n)
for i = 1:n
    b_i       = x\Re[:,i]                #OLS
    residual  = Re[:,i] - x*b_i
    b[:,i]    = b_i[2:end]
    VarRes[i] = var(residual)
end    
println("\nOLS coefficients, regressing Re on [constant, Rme, RSMB, RHML] ")
printmat(round(b,3)) 
#-------------------------------------------------------------------------

Omega = cov(x[:,2:end])      #covariance matrix of factors
CovR = fill(NaN,(n,n))
for i = 1:n    
    for j = 1:n
        CovR[i,j] = (b[:,i]'Omega*b[:,j])[1]
        if i == j
            CovR[i,i] = CovR[i,i] + VarRes[i]
        end
    end    
end

println("\nCovariance matrix calculated from betas: ")
printmat(round(CovR,2))

println("\nCovariance matrix calculated from data: ")
printmat(round(cov(Re),2))


OLS coefficients, regressing Re on [constant, Rme, RSMB, RHML] 
     1.070     1.080     1.035     1.056     1.041
     1.264     0.768     0.437     0.153    -0.088
    -0.278     0.160     0.487     0.603     0.770


Covariance matrix calculated from betas: 
    73.480    41.850    31.050    25.450    18.940
    41.850    37.370    27.380    24.140    20.150
    31.050    27.380    26.800    22.240    20.110
    25.450    24.140    22.240    25.200    20.720
    18.940    20.150    20.110    20.720    25.330


Covariance matrix calculated from data: 
    73.480    43.020    29.470    25.010    20.190
    43.020    37.370    29.070    25.140    21.410
    29.470    29.070    26.800    23.630    20.980
    25.010    25.140    23.630    25.200    21.530
    20.190    21.410    20.980    21.530    25.330

